# Задание 2.2 - Введение в PyTorch

Для этого задания потребуется установить версию PyTorch 1.0

https://pytorch.org/get-started/locally/

В этом задании мы познакомимся с основными компонентами PyTorch и натренируем несколько небольших моделей.
GPU нам пока не понадобится.

Основные ссылки:
https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html
https://pytorch.org/docs/stable/nn.html
https://pytorch.org/docs/stable/torchvision/index.html

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as dset
from torch.utils.data.sampler import SubsetRandomSampler, Sampler

from torchvision import transforms

import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np

In [2]:
# First, lets load the dataset
data_train = dset.SVHN('./data/', split='train',
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize(mean=[0.43,0.44,0.47],
                                               std=[0.20,0.20,0.20])                           
                       ])
                      )
data_test = dset.SVHN('./data/', split='test', 
                      transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize(mean=[0.43,0.44,0.47],
                                               std=[0.20,0.20,0.20])                           
                       ]))

Теперь мы разделим данные на training и validation с использованием классов SubsetRandomSampler и DataLoader.

DataLoader подгружает данные, предоставляемые классом Dataset, во время тренировки и группирует их в батчи. Он дает возможность указать Sampler, который выбирает, какие примеры из датасета использовать для тренировки. Мы используем это, чтобы разделить данные на training и validation.

Подробнее: https://pytorch.org/tutorials/beginner/data_loading_tutorial.html

In [3]:
batch_size = 64

data_size = data_train.data.shape[0]
validation_split = .2
split = int(np.floor(validation_split * data_size))
indices = list(range(data_size))
np.random.shuffle(indices)

train_indices, val_indices = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, 
                                           sampler=train_sampler)
val_loader = torch.utils.data.DataLoader(data_train, batch_size=batch_size,
                                         sampler=val_sampler)

В нашей задаче мы получаем на вход изображения, но работаем с ними как с одномерными массивами. Чтобы превратить многомерный массив в одномерный, мы воспользуемся очень простым вспомогательным модулем Flattener.

In [4]:
sample, label = data_train[0]
print("SVHN data sample shape: ", sample.shape)
# As you can see, the data is shaped like an image

# We'll use a special helper module to shape it into a tensor
class Flattener(nn.Module):
    def forward(self, x):
        batch_size, *_ = x.shape
        return x.view(batch_size, -1)

SVHN data sample shape:  torch.Size([3, 32, 32])


И наконец, мы создаем основные объекты PyTorch:

nn_model - собственно, модель с нейросетью

loss - функцию ошибки, в нашем случае CrossEntropyLoss

optimizer - алгоритм оптимизации, в нашем случае просто SGD

In [5]:
nn_model = nn.Sequential(
            Flattener(),
            nn.Linear(3*32*32, 100),
            nn.ReLU(inplace=True),
            nn.Linear(100, 10), 
         )
nn_model.type(torch.FloatTensor)

# We will minimize cross-entropy between the ground truth and
# network predictions using an SGD optimizer
loss = nn.CrossEntropyLoss().type(torch.FloatTensor)
optimizer = optim.SGD(nn_model.parameters(), lr=1e-2, weight_decay=1e-1)

# Тренируем!

Ниже приведена функция train_model, реализующая основной цикл тренировки PyTorch.

Каждую эпоху эта функция вызывает функцию compute_accuracy, которая вычисляет точность на validation, эту последнюю функцию предлагается реализовать вам.

In [35]:
# This is how to implement the same main train loop in PyTorch. Pretty easy, right?

def train_model(model, train_loader, val_loader, loss, optimizer, num_epochs):    
    loss_history = []
    train_history = []
    val_history = []
    for epoch in range(num_epochs):
        model.train() # Enter train mode
        
        loss_accum = 0
        correct_samples = 0
        total_samples = 0
        for i_step, (x, y) in enumerate(train_loader):
            prediction = model(x)    
            loss_value = loss(prediction, y)
            optimizer.zero_grad()
            loss_value.backward()
            optimizer.step()
            
            _, indices = torch.max(prediction, 1)
            correct_samples += torch.sum(indices == y)
            total_samples += y.shape[0]
            
            loss_accum += loss_value

        ave_loss = loss_accum / (i_step + 1)
        train_accuracy = float(correct_samples) / total_samples
        val_accuracy = compute_accuracy(model, val_loader)
        
        loss_history.append(float(ave_loss))
        train_history.append(train_accuracy)
        val_history.append(val_accuracy)
        
        print("Average loss: %f, Train accuracy: %f, Val accuracy: %f" % (ave_loss, train_accuracy, val_accuracy))
        
    return loss_history, train_history, val_history
        
def compute_accuracy(model, loader):
    """
    Computes accuracy on the dataset wrapped in a loader
    
    Returns: accuracy as a float value between 0 and 1
    """
    model.eval() # Evaluation mode
    correct_samples = 0
    total_samples = 0         
    for i_step, (x, y) in enumerate(val_loader):
        count = i_step
        predictions = nn_model(x)
        pred = torch.argmax(predictions, axis=1)
        correct_samples += torch.sum(pred == y)
        total_samples += y.shape[0]

    val_accuracy = float(correct_samples) / total_samples
    # print("val accuracy is ", val_accuracy)
        

    return val_accuracy


    
    # TODO: Implement the inference of the model on all of the batches from loader,
    #       and compute the overall accuracy.
    # Hint: PyTorch has the argmax function!
    
    raise Exception("Not implemented")
    
    return 0

# val_loader.
loss_history, train_history, val_history = train_model(nn_model, train_loader, val_loader, loss, optimizer, 3)

Average loss: 1.307358, Train accuracy: 0.657168, Val accuracy: 0.646236
Average loss: 1.306368, Train accuracy: 0.657134, Val accuracy: 0.643642
Average loss: 1.306151, Train accuracy: 0.658960, Val accuracy: 0.642072


In [34]:
# count = 0
            # _, indices = torch.max(prediction, 1)
            # correct_samples += torch.sum(indices == y)
            # total_samples += y.shape[0]

correct_samples = 0
total_samples = 0         
for i_step, (x, y) in enumerate(val_loader):
    count = i_step
    predictions = nn_model(x)
    pred = torch.argmax(predictions, axis=1)
    correct_samples += torch.sum(pred == y)
    total_samples += y.shape[0]
    # print(y.shape)
    # print(i_step)

val_accuracy = float(correct_samples) / total_samples
print(val_accuracy)
# print(count)
# val_loader.dataset.data
# print(y.shape)
# print(data_train.data.shape)

# all_y = val_loader.dataset.labels
# all_X = val_loader.dataset.data
# # nn_model.eval() # Evaluation mode
# predictions = nn_model(all_X[0])
# print("pred shape ", all_X.shape)

0.6326530612244898
